##  Use panel data for regression 

In [15]:
import pandas as pd 
# Display 6 columns for viewing purposes
pd.set_option('display.max_columns', 20)

# Reduce decimal points to 2
pd.options.display.float_format = '{:,.2f}'.format



DataKnowledgeEconomyPaper = pd.read_excel('DataKnowledgeEconomyPaper.xls')

In [16]:
DataKnowledgeEconomyPaper.head()

,cn,year,LM Income,M Income,UM Income,L Income,English,French,Christians,Islam,...,Tel,Internet,PSE,SSE,TSE,Educatex,ICTex,Innovex,Instireg,Creditex
0,Algeria,1996,0,1,1,0,0,1,0,1,...,4.44,0.00,100.75,60.16,11.90,1.04,-0.60,-0.08,-1.57,0.19
1,Algeria,1997,0,1,1,0,0,1,0,1,...,4.79,0.01,101.99,60.51,nan,nan,-0.57,-0.11,nan,0.16
2,Algeria,1998,0,1,1,0,0,1,0,1,...,4.98,0.02,nan,nan,nan,nan,-0.55,-0.08,-2.15,0.15
3,Algeria,1999,0,1,1,0,0,1,0,1,...,5.32,0.20,105.53,nan,14.24,nan,-0.50,-0.04,nan,0.14
4,Algeria,2000,0,1,1,0,0,1,0,1,...,5.77,0.49,107.81,nan,nan,nan,-0.43,0.00,-1.53,0.13


In [11]:
#DataKnowledgeEconomyPaper.to_panel()

In [12]:
DataKnowledgeEconomyPaper.columns

Index(['cn', 'year', 'LM Income', 'M Income', 'UM Income', 'L Income ',
       'English ', 'French ', 'Christians', 'Islam', 'Landlocked ',
       'Notlandlocked', 'Oil', 'Nonoil', 'Conflicts', 'Nonconflicts', 'SSA',
       'NorthAfri', 'pcrdbofgdp', 'CC', 'GE', 'PolSta', 'R.Q', 'R.L', 'VA',
       'IRS', 'STJA', 'Totpatent', 'Trademark', 'Mobile', 'Tel', 'Internet',
       'PSE', 'SSE', 'TSE', 'Educatex', 'ICTex', 'Innovex', 'Instireg',
       'Creditex'],
      dtype='object')

In [19]:
#real = DataKnowledgeEconomyPaper.pivot_table(values='LM Income', index='year',columns=['cn', 'M Income', 'TSE'])

In [21]:
#real

In [5]:
p = pd.Panel(DataKnowledgeEconomyPaper)
print (p)

/Users/garbamoussa/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)


ValueError: Panel constructor not properly called!

In [ ]:
#Generate some normally distributed: data (x), fixed effect (u), and shock (e).
import numpy
T = 100
N = 1000
x = numpy.random.normal(5,1,(T,N))
u = numpy.random.normal(2,1,(1,N))
e = numpy.random.normal(0,1,(T,N))

In [2]:
#Define y as a simple linear equation of lagged y, x, u and e
y = numpy.zeros((T,N))
for t in range(1,T):
    y[t] = 0.6*y[t-1] + 2*x[t] + u[0] + e[t]

In [3]:
#Put the last 10 periods of data into a data frame
import pandas
#Put y and x into pandas series by stacking the 2D data
sy = pandas.DataFrame(y[T-10:]).stack()
sx = pandas.DataFrame(x[T-10:]).stack()

#now combine into one data frame and label the time and location dimensions
df = pandas.DataFrame(dict(y = sy, x = sx))
df.index = df.index.set_names(['T','N'])
df = df.swaplevel(0,1).sort_index()

#Now convert the fixed effect to a dataframe varaible
du = pandas.DataFrame(dict(u=u[0]))
du.index = du.index.set_names('N')

#And merge it into the dataframe
df = df.reset_index().merge(du.reset_index(),on = 'N', how = 'left').set_index(['N','T'])

#Add the fixed effects
fe = []
for g,d in df.groupby(level = 'N'):
    df['fe_%i'%g] = 0    
    df.loc[df.index.get_level_values('N')==g,'fe_%i'%g] = 1
    fe.append('fe_%i'%g)
    
#And the lags of y
df['lagy'] = df.groupby(level = 'N')['y'].shift(1)
df

x          y         u  fe_0  fe_1  fe_2  fe_3  fe_4  fe_5  \
N   T                                                                      
0   0  4.444952  28.714711  2.260975     1     0     0     0     0     0   
    1  4.933449  29.991171  2.260975     1     0     0     0     0     0   
    2  5.689179  31.051672  2.260975     1     0     0     0     0     0   
    3  4.752026  30.576453  2.260975     1     0     0     0     0     0   
    4  5.255060  32.112165  2.260975     1     0     0     0     0     0   
    5  3.517943  27.916395  2.260975     1     0     0     0     0     0   
    6  5.167789  29.670215  2.260975     1     0     0     0     0     0   
    7  4.976900  29.933762  2.260975     1     0     0     0     0     0   
    8  5.946123  31.514391  2.260975     1     0     0     0     0     0   
    9  3.791215  28.211379  2.260975     1     0     0     0     0     0   
1   0  5.796742  31.445713  1.849825     0     1     0     0     0     0   
    1  6.707199  34.599125  1.849825     0     1     0     0     0     0   
    2  4.613466  31.829489  1.849825     0     1     0     0     0     0   
    3  3.118445  25.760386  1.849825     0     1     0     0     0     0   
    4  3.472556  24.211279  1.849825     0     1     0     0     0     0   
    5  5.041232  26.145877  1.849825     0     1     0     0     0     0   
    6  3.262936  24.768325  1.849825     0     1     0     0     0     0   
    7  4.339988  24.050762  1.849825     0     1     0     0     0     0   
    8  4.920299  25.384908  1.849825     0     1     0     0     0     0   
    9  4.322684  25.554173  1.849825     0     1     0     0     0     0   
2   0  5.876023  28.457695  1.561547     0     0     1     0     0     0   
    1  2.497742  24.437828  1.561547     0     0     1     0     0     0   
    2  5.718350  26.879440  1.561547     0     0     1     0     0     0   
    3  5.696182  29.462914  1.561547     0     0     1     0     0     0   
    4  5.506474  31.641740  1.561547     0     0     1     0     0     0   
    5  2.872319  27.227434  1.561547     0     0     1     0     0     0   
    6  2.651272  23.813824  1.561547     0     0     1     0     0     0   
    7  6.758578  28.847863  1.561547     0     0     1     0     0     0   
    8  5.365327  29.000894  1.561547     0     0     1     0     0     0   
    9  4.272929  28.110917  1.561547     0     0     1     0     0     0   
...         ...        ...       ...   ...   ...   ...   ...   ...   ...   
997 0  7.020752  35.550959  2.274496     0     0     0     0     0     0   
    1  6.275147  36.135772  2.274496     0     0     0     0     0     0   
    2  5.200175  33.735789  2.274496     0     0     0     0     0     0   
    3  4.470738  32.902570  2.274496     0     0     0     0     0     0   
    4  5.465859  34.074428  2.274496     0     0     0     0     0     0   
    5  3.986207  31.058637  2.274496     0     0     0     0     0     0   
    6  5.638014  33.345093  2.274496     0     0     0     0     0     0   
    7  4.045045  30.087318  2.274496     0     0     0     0     0     0   
    8  5.932176  33.732849  2.274496     0     0     0     0     0     0   
    9  5.174276  35.191079  2.274496     0     0     0     0     0     0   
998 0  4.788774  35.991495  3.899248     0     0     0     0     0     0   
    1  6.886087  40.745509  3.899248     0     0     0     0     0     0   
    2  3.569630  35.529512  3.899248     0     0     0     0     0     0   
    3  3.820989  33.131542  3.899248     0     0     0     0     0     0   
    4  4.590642  32.036185  3.899248     0     0     0     0     0     0   
    5  5.866091  34.222636  3.899248     0     0     0     0     0     0   
    6  6.247055  37.294303  3.899248     0     0     0     0     0     0   
    7  4.743888  35.860204  3.899248     0     0     0     0     0     0   
    8  4.270725  33.045748  3.899248     0     0     0     0     0     0   
    9  5.627130  33.506113  3.899248     0     0     0     0  

In [4]:
#Run the regression of y on x and lagged y without fixed effects demonstrating the bias of OLS
import statsmodels.api as sm
reg = df.dropna()
sm.OLS(reg['y'],sm.add_constant(reg[['lagy','x']])).fit().summary()

c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.882
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                 3.375e+04
Date:                Wed, 28 Jun 2017   Prob (F-statistic):               0.00
Time:                        17:41:36   Log-Likelihood:                -14774.
No. Observations:                9000   AIC:                         2.955e+04
Df Residuals:                    8997   BIC:                         2.957e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1807      0.129    -24.746      0.000      -3.433      -2.929
lagy           0.7737      0.004    214.721      0.000       0.767       0.781
x              1.9959      0.013    151.618      0.000       1.970       2.022
==============================================================================
Omnibus:                        2.657   Durbin-Watson:                   1.663
Prob(Omnibus):                  0.265   Jarque-Bera (JB):                2.669
Skew:                          -0.042   Prob(JB):                        0.263
Kurtosis:                       2.989   Cond. No.                         300.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
reg

x          y         u  fe_0  fe_1  fe_2  fe_3  fe_4  fe_5  \
N   T                                                                      
0   1  4.933449  29.991171  2.260975     1     0     0     0     0     0   
    2  5.689179  31.051672  2.260975     1     0     0     0     0     0   
    3  4.752026  30.576453  2.260975     1     0     0     0     0     0   
    4  5.255060  32.112165  2.260975     1     0     0     0     0     0   
    5  3.517943  27.916395  2.260975     1     0     0     0     0     0   
    6  5.167789  29.670215  2.260975     1     0     0     0     0     0   
    7  4.976900  29.933762  2.260975     1     0     0     0     0     0   
    8  5.946123  31.514391  2.260975     1     0     0     0     0     0   
    9  3.791215  28.211379  2.260975     1     0     0     0     0     0   
1   1  6.707199  34.599125  1.849825     0     1     0     0     0     0   
    2  4.613466  31.829489  1.849825     0     1     0     0     0     0   
    3  3.118445  25.760386  1.849825     0     1     0     0     0     0   
    4  3.472556  24.211279  1.849825     0     1     0     0     0     0   
    5  5.041232  26.145877  1.849825     0     1     0     0     0     0   
    6  3.262936  24.768325  1.849825     0     1     0     0     0     0   
    7  4.339988  24.050762  1.849825     0     1     0     0     0     0   
    8  4.920299  25.384908  1.849825     0     1     0     0     0     0   
    9  4.322684  25.554173  1.849825     0     1     0     0     0     0   
2   1  2.497742  24.437828  1.561547     0     0     1     0     0     0   
    2  5.718350  26.879440  1.561547     0     0     1     0     0     0   
    3  5.696182  29.462914  1.561547     0     0     1     0     0     0   
    4  5.506474  31.641740  1.561547     0     0     1     0     0     0   
    5  2.872319  27.227434  1.561547     0     0     1     0     0     0   
    6  2.651272  23.813824  1.561547     0     0     1     0     0     0   
    7  6.758578  28.847863  1.561547     0     0     1     0     0     0   
    8  5.365327  29.000894  1.561547     0     0     1     0     0     0   
    9  4.272929  28.110917  1.561547     0     0     1     0     0     0   
3   1  5.218778  24.459751 -0.737996     0     0     0     1     0     0   
    2  3.596131  22.542170 -0.737996     0     0     0     1     0     0   
    3  6.842888  27.415825 -0.737996     0     0     0     1     0     0   
...         ...        ...       ...   ...   ...   ...   ...   ...   ...   
996 7  4.288911  32.648000  3.397754     0     0     0     0     0     0   
    8  6.214891  35.124946  3.397754     0     0     0     0     0     0   
    9  4.699480  33.739724  3.397754     0     0     0     0     0     0   
997 1  6.275147  36.135772  2.274496     0     0     0     0     0     0   
    2  5.200175  33.735789  2.274496     0     0     0     0     0     0   
    3  4.470738  32.902570  2.274496     0     0     0     0     0     0   
    4  5.465859  34.074428  2.274496     0     0     0     0     0     0   
    5  3.986207  31.058637  2.274496     0     0     0     0     0     0   
    6  5.638014  33.345093  2.274496     0     0     0     0     0     0   
    7  4.045045  30.087318  2.274496     0     0     0     0     0     0   
    8  5.932176  33.732849  2.274496     0     0     0     0     0     0   
    9  5.174276  35.191079  2.274496     0     0     0     0     0     0   
998 1  6.886087  40.745509  3.899248     0     0     0     0     0     0   
    2  3.569630  35.529512  3.899248     0     0     0     0     0     0   
    3  3.820989  33.131542  3.899248     0     0     0     0     0     0   
    4  4.590642  32.036185  3.899248     0     0     0     0     0     0   
    5  5.866091  34.222636  3.899248     0     0     0     0     0     0   
    6  6.247055  37.294303  3.899248     0     0     0     0     0     0   
    7  4.743888  35.860204  3.899248     0     0     0     0     0     0   
    8  4.270725  33.045748  3.899248     0     0     0     0  

In [6]:
#Now we run the regression of y on x and lagged y and fixed effects, demonstrating the bias of FE OLS
sm.OLS(reg['y'],reg[['lagy','x']+fe]).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     112.8
Date:                Wed, 28 Jun 2017   Prob (F-statistic):               0.00
Time:                        17:41:41   Log-Likelihood:                -12183.
No. Observations:                9000   AIC:                         2.637e+04
Df Residuals:                    7998   BIC:                         3.349e+04
Df Model:                        1001                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
lagy           0.5554      0.005    118.204      0.000       0.546       0.565
x              1.9777      0.011    176.312      0.000       1.956       2.000
fe_0           3.6792      0.368      9.987      0.000       2.957       4.401
fe_1           2.8600      0.362      7.891      0.000       2.150       3.571
fe_2           3.2154      0.363      8.854      0.000       2.503       3.927
fe_3           0.9131      0.358      2.548      0.011       0.211       1.616
fe_4           3.4473      0.371      9.280      0.000       2.719       4.175
fe_5           1.6217      0.362      4.484      0.000       0.913       2.331
fe_6           3.3376      0.366      9.118      0.000       2.620       4.055
fe_7           2.5003      0.362      6.900      0.000       1.790       3.211
fe_8           1.6900      0.359      4.705      0.000       0.986       2.394
fe_9           3.1371      0.370      8.471      0.000       2.411       3.863
fe_10          4.5622      0.370     12.324      0.000       3.837       5.288
fe_11          2.4390      0.367      6.653      0.000       1.720       3.158
fe_12          3.1584      0.365      8.661      0.000       2.444       3.873
fe_13          4.4932      0.379     11.842      0.000       3.749       5.237
fe_14          4.6533      0.379     12.282      0.000       3.911       5.396
fe_15          3.3228      0.368      9.024      0.000       2.601       4.045
fe_16          5.9118      0.385     15.361      0.000       5.157       6.666
fe_17          3.2718      0.364      8.986      0.000       2.558       3.986
fe_18          1.7610      0.368      4.787      0.000       1.040       2.482
fe_19          3.6802      0.373      9.876      0.000       2.950       4.411
fe_20          3.7011      0.373      9.923      0.000       2.970       4.432
fe_21          3.3131      0.369      8.985      0.000       2.590       4.036
fe_22          3.1423      0.362      8.676      0.000       2.432       3.852
fe_23          3.3562      0.369      9.108      0.000       2.634       4.079
fe_24          4.3244      0.375     11.521      0.000       3.589       5.060
fe_25          3.4215      0.370      9.236      0.000       2.695       4.148
fe_26          2.4333      0.366      6.656      0.000       1.717       3.150
fe_27          2.1925      0.362      6.065      0.000       1.484       2.901
fe_28          2.3169      0.362      6.395      0.000       1.607       3.027
fe_29          2.1382      0.364      5.867      0.000       1.424       2.853
fe_30          2.3892      0.372      6.422      0.000       1.660       3.119
fe_31          3.8734      0.362     10.685      0.000       3.163       4.584
fe_32          2.1452      0.361      5.946      0.000       1.438       2.852
fe_33          4.5567      0.371     12.271      0.000       3.829       5.285
fe

In [7]:
#Now we'll run the model suggested by Arellano and Bond, which uses an Instrumental variables approach to 
#produce unbiased estimates
from linearmodels import IV2SLS, IVLIML, IVGMM, IVGMMCUE
class PanelLaggedDep:
    '''
    Estimates values of rho and beta in the following Arrelano Bond model on a panel data set:
    
    y(i,t) = rho*y(i,t-1) + beta*x(i,t) + u(i) + e(i,t)
    
    Where y is the observed endogenous variable (scalar), x is an observed vector of exogenous variables, 
    u is an unobserved individual effect, and e(i,t) is independent across i and t.
    
    Fixed effects OLS will usually result in biased estimates of rho because of the correlation between 
    y(i,t) and e(i,t) stemming from u(i). The Arrelano-Bond procedure first differences the observables 
    and then uses lags of y as instruments.
    
    The `endog` and `exog` vars are assumed to have a multiindex structure with the first level identifying 
    cross-section and the second identifying time. The time level must be sorted in ascending order. The
    panel data can have missing values and be unbalanced (i.e., different sample size for each group).
    
    `lags` is the number of lags of the `endog` series. Untested for anything other than lags==1
    '''
    def __init__(self, endog, exogs, lags=1, systemGMM = False):
        min_t = min(endog.index.get_level_values(1)) #starting period
        T = max(endog.index.get_level_values(1)) + 1 - min_t #total number of periods
        
        #Names for the original and differenced endog and exog vars
        ename = endog.name
        Dename = 'D' + ename
        xnames = exogs.columns.tolist()
        Dxnames = ['D'+x for x in xnames]

        #We'll store all of the data in a dataframe
        self.data = pandas.DataFrame()
        self.data[ename] = endog
        self.data[Dename] = endog.groupby(level=0).diff()

        #Generate and store the lags of the differenced endog variable
        Lenames = []
        LDenames = []
        for k in range(1,lags+1):
            col = 'L%s%s'%(k,ename)
            colD = 'L%s%s'%(k,Dename)
            Lenames.append(col)
            LDenames.append(colD)
            self.data[col] = self.data[ename].shift(k)
            self.data[colD] = self.data[Dename].shift(k)
        
        #Store the original and the diffs of the exog variables
        for x in xnames:
            self.data[x] = exogs[x]
            self.data['D'+x] = exogs[x].groupby(level=0).diff()

        #Set up the instruments -- lags of the endog levels for different time periods
        instrnames = []
        for t in range(lags+1, T): #TODO: Check this -- works for lags == 1, not sure if for anything else
            for k in range(lags+1, t+1):
                col = 'ILVL_t%iL%i'%(t+min_t,k)
                instrnames.append(col)
                self.data[col] = endog.groupby(level=0).shift(k)
                self.data.loc[endog.index.get_level_values(1) != t+min_t, col] = 0

        if systemGMM:
            #With the systems GMM estimator we have additional instruments of lagged differences
            instrnamessys = []
            for t in range(lags, T): #TODO: Check this -- works for lags == 1, not sure if for anything else
                col = 'IDIFF_t%iL'%t
                instrnamessys.append(col)
                self.data[col] = self.data[LDenames[0]].groupby(level=0).shift(lags)
                self.data.loc[endog.index.get_level_values(1) != t+min_t, col] = 0

            #Then to estimate the system GMM we stack differenced and undifferenced data and their corresponding instruments
            cols1 = [Dename]+LDenames+Dxnames+instrnames #variables used for the differences part of the regression
            cols2 = [ename]+Lenames+xnames+instrnamessys #variable used for the levels part of the regression
            cols2R = [Dename]+LDenames+Dxnames+instrnamessys #used to rename the variables that we want to overlap in the system reg.

            #The differenced data set
            data1 = self.data[cols1].copy()
            for c in instrnamessys:
                data1[c] = 0 #zero out the instruments that apply to undifferenced data
            
            #The undifferenced data set
            data2 = self.data[cols2].copy()
            data2.columns = cols2R
            for c in instrnames:
                data2[c] = 0 #zero out the instruments that apply to differenced data
            
            #Add an index level to each data series so we can join without creating duplicate index values
            a1 = data1.index.get_level_values(0)
            a2 = data1.index.get_level_values(1)
            n = len(a1)
            data1.index = pandas.MultiIndex.from_arrays([[1]*n,a1,a2])
            data2.index = pandas.MultiIndex.from_arrays([[2]*n,a1,a2])
            
            #Now append the series together
            self.data = data1.append(data2)
                
        dropped = self.data.dropna()
        dropped['CLUSTER_VAR'] = dropped.index.get_level_values(0)
        self.model = IVGMM(dropped[Dename], dropped[Dxnames], dropped[LDenames], dropped[instrnames], weight_type='clustered', clusters = dropped['CLUSTER_VAR'])

    def fit(self):
        return self.model.fit()

preg = reg.reset_index().set_index(['N','T'])
mod = PanelLaggedDep(preg['y'], preg[['x']], lags=1, systemGMM = False)
mod.fit().summary

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-GMM Estimation Summary                           
==============================================================================
Dep. Variable:                     Dy   R-squared:                      0.6793
Estimator:                     IV-GMM   Adj. R-squared:                 0.6792
No. Observations:                7000   F-statistic:                 1.309e+04
Date:                Wed, Jun 28 2017   P-value (F-stat)                0.0000
Time:                        17:41:45   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Dx             2.0078     0.0324     61.992     0.0000      1.9443      2.0713
L1Dy           0.6204     0.0314     19.764     0.0000      0.5589      0.6820
==============================================================================

Endogenous: L1Dy
Instruments: ILVL_t3L2, ILVL_t4L2, ILVL_t4L3, ILVL_t5L2, ILVL_t5L3, ILVL_t5L4, ILVL_t6L2, ILVL_t6L3, ILVL_t6L4, ILVL_t6L5, ILVL_t7L2, ILVL_t7L3, ILVL_t7L4, ILVL_t7L5, ILVL_t7L6, ILVL_t8L2, ILVL_t8L3, ILVL_t8L4, ILVL_t8L5, ILVL_t8L6, ILVL_t8L7, ILVL_t9L2, ILVL_t9L3, ILVL_t9L4, ILVL_t9L5, ILVL_t9L6, ILVL_t9L7, ILVL_t9L8
GMM Covariance
Debiased: False
Robust (Heteroskedastic)
"""